In [31]:
#import all libraries
import pandas as pd
import requests
import bs4
import re
import zipfile
import logging
import math
from time import sleep
from concurrent.futures import ThreadPoolExecutor
from concurrent.futures import as_completed

In [32]:
def get_links():
    urls = list(pd.read_csv('books_url.csv')['link'])
    return urls

In [33]:
def get_soup(input_url):
    headers = {
        'User-Agent': 'My User Agent 1.0',
        "Accept-Language": "en-US,en;q=0.5"
    }
    response = requests.get(input_url, headers=headers)
    if response.status_code != 200:
        print("Error in getting link")
        print("response code is : ", response.status_code)
    soup = bs4.BeautifulSoup(response.text, 'html.parser')
    return soup

In [34]:
def get_fa_title(soup):
    title = soup.select('.product-name strong')[0]
    return title.text


In [35]:
def get_en_title(soup):
    title = soup.select('.product-name-englishname')[0]
    return title.text

In [36]:
def get_price(soup):
    price = soup.select('.price-broken , .col-md-7 .price:nth-child(1)')[0].text
    return int(price.replace(',', ''))  #TODO make the price integer

In [37]:
def get_discount(soup):
    try:
        discount_price = int(soup.select('.col-md-12+ .clearfix .price-special')[0].text.replace(',', ''))
        discount_price = ((get_price(soup) - discount_price) / get_price(soup)) * 100
    except Exception:
        discount_price = 0
        logging.exception("This book has no discount!")
    return discount_price  

In [38]:
def get_score(soup):
    soup = soup.find('div', {'class':'col-md-7'}).find('li', {'class':'pull-left'}).find('div', {'class':'my-rating'})
    soup_str = str(soup)

    match = re.search(r'data-rating="(\d+\.\d+)"', soup_str)
    if match:
        data_rating = match.group(1)
        return data_rating
    

In [39]:
def get_publisher(soup):
    try:
        publisher = soup.select('.prodoct-attribute-items:nth-child(1) a .prodoct-attribute-item')[0].text.strip()
    except Exception:
        publisher = None
        logging.exception("This book has no publisher!")
    return publisher

In [71]:
def get_author(soup):
    try:
        author = soup.select('.prodoct-attribute-items+ .prodoct-attribute-items a .prodoct-attribute-item')[0].text.strip()
    except Exception:
        author = None
        logging.exception("This book has no author!")    
    return author

In [41]:
def get_code(soup):
    code = soup.select('tr:nth-child(1) td+ td')[0]
    return code.text

In [42]:
def get_isbn(soup):
    isbn = soup.select('.product-table .ltr')[0]
    cleaned_isbn = isbn.text.strip()
    cleaned_isbn = re.sub('[^0-9-]', '', cleaned_isbn)
    return cleaned_isbn

In [43]:
def get_pages(soup):
    pages = soup.select('tr:nth-child(5) .rtl')[0]
    cleaned_pages = pages.text.strip()
    cleaned_pages = re.sub('[^0-9-]', '', cleaned_pages)
    return cleaned_pages

In [44]:
def get_per_date(soup):
    try:
        date = soup.select('.product-table > tbody:nth-child(1) > tr:nth-child(6) > td:nth-child(2)')[0].text
    except Exception:
        date = None
        logging.exception("This book has persian calendar!")
    return date.strip() 

In [45]:
def get_ad_date(soup):
    pass #TODO
    return None

In [46]:
def get_size(soup):
    size = soup.select(
        'html body div.container div div.product-container.well.clearfix div.clearfix div.col-md-9.col-sm-9 div.row div.col-md-5 table.product-table tbody tr td.rtl')[0].text
    return size.strip()


In [47]:
def get_material(soup):
    pass #TODO
    return None


In [48]:
def get_series(soup):
    pass #TODO
    return None


In [49]:
def get_send_time(soup):
    pass
    return None


In [50]:
def get_summary(soup):
    try:
        summary = soup.select('.product-description')[0].text.strip()
    except Exception:
        summary = None
        logging.exception("This book has no summary!")
    return summary

In [51]:
def get_tags(soup):
    tags = soup.select('.product-tags-item')
    tags_list = []
    for tag in tags:
        tags_list += [tag.text.strip()]
    return tags_list

In [69]:
 def get_book_detail(book_soup):
    book_fa_title = get_fa_title(book_soup)
    book_en_title = get_en_title(book_soup)
    book_price = get_price(book_soup)
    book_discount_percent = get_discount(book_soup)
    book_score = get_score(book_soup)
    book_publisher = get_publisher(book_soup)
    book_author = get_author(book_soup)
    book_code = get_code(book_soup)
    book_Isbn = get_isbn(book_soup)
    book_pages = get_pages(book_soup)
    book_publication_per_date = get_per_date(book_soup)
    book_publication_ad_date = get_ad_date(book_soup)
    book_size = get_size(book_soup)
    book_cover_material = get_material(book_soup)
    book_print_series = get_series(book_soup)
    book_earliest_send_time = get_send_time(book_soup)
    book_summary = get_summary(book_soup)
    book_tags = get_tags(book_soup)
    
    book_data = [book_fa_title, book_en_title, book_price, book_discount_percent, book_score, book_publisher, book_author,
            book_code, book_Isbn, book_pages, book_publication_per_date, book_publication_ad_date, book_size,
            book_cover_material, book_print_series, book_earliest_send_time, book_summary, book_tags]  #TODO
    return book_data

In [74]:
data_list = []
links = get_links()[:10] + ['https://www.iranketab.ir/book/270-gone-with-the-wind']

for link in links:
    try:
        site_soup = get_soup(link)
        site_page_books = site_soup.select('.clearfix .clearfix .row')
        for book_index in range(0, len(site_page_books), 2):
            data = get_book_detail(site_page_books[book_index])
            data_list += [data];
    except Exception:
        logging.exception("An error occurred");

ERROR:root:This book has no summary!
Traceback (most recent call last):
  File "C:\Users\raeim\AppData\Local\Temp\ipykernel_4956\1115907846.py", line 3, in get_summary
    summary = soup.select('.product-description')[0].text.strip()
IndexError: list index out of range
ERROR:root:This book has no summary!
Traceback (most recent call last):
  File "C:\Users\raeim\AppData\Local\Temp\ipykernel_4956\1115907846.py", line 3, in get_summary
    summary = soup.select('.product-description')[0].text.strip()
IndexError: list index out of range
ERROR:root:This book has no summary!
Traceback (most recent call last):
  File "C:\Users\raeim\AppData\Local\Temp\ipykernel_4956\1115907846.py", line 3, in get_summary
    summary = soup.select('.product-description')[0].text.strip()
IndexError: list index out of range
ERROR:root:This book has no summary!
Traceback (most recent call last):
  File "C:\Users\raeim\AppData\Local\Temp\ipykernel_4956\1115907846.py", line 3, in get_summary
    summary = soup.sel

In [75]:
tableOfData = pd.DataFrame(data_list,
                           columns=['fa_title', 'en_title', 'price', 'discount', 'score', 'publisher', 'author',
                                    'code', 'Isbn', 'pages', 'publication_per_date', 'publication_ad_date', 'size',
                                    'cover_material', 'print_series', 'earliest_send_time', 'summary', 'tags'])
tableOfData

,fa_title,en_title,price,discount,score,publisher,author,code,Isbn,pages,publication_per_date,publication_ad_date,size,cover_material,print_series,earliest_send_time,summary,tags
0,کتاب کلیات پزشکی سنتی چینی و طب سوزنی,Pezeshki Sonnati,300000,15.0,3.87,المعی,حسن رضوانی,70554,978-9647553261,1402,شومیز,None,وزیری,None,None,None,None,[]
1,کتاب بانوان عمارت میسالونگی,The Ladies of Missalonghi,30000,15.0,3.17,روشنگران و مطالعات زنان,کالین مک کالو,51563,978-9641940289,184,1388,None,رقعی,None,None,None,None,[]
2,کتاب گوزن شاخدار فایده اش چیه؟,What Use Is A Moose?,70000,15.0,3.58,علمی و فرهنگی,مارتین وادل,49138,978-9644458071,32,1400,None,وزیری,None,None,None,None,[]
3,کتاب تانیا,Tanya,75000,20.0,3.62,پژواک فرزان,آلکسی آربوزوف,116421,978-6009313884,159,1396,None,جیبی,None,None,None,None,[]
4,کتاب دانه های روشنایی,Flakes of light,9000,30.0,3.67,اشک,عباس مهرپویا,76016,978-9642683864,1388,شومیز,None,خشتی,None,None,None,None,[]
5,کتاب رازهای ناگفته ی بازاریابی,Marketing,20000,15.0,3.67,نیما عربشاهی,جو ویتالی,73891,978-9640669687,96,1384,None,خشتی,None,None,None,None,[]
6,کتاب سلام بر عاشورا,Salam bar Ashoora,5000,15.0,3.63,اطلاعات,رضا اسماعیلی,41857,978-9644239441,1394,شومیز,None,رقعی,None,None,None,None,[]
7,کتاب قاصدک ها در هوا ایستاده اند,Dandelions are standing in the air,69000,20.0,3.18,دنیای نو,مهدی حسینی (مهد),107977,978-9641721932,1401,شومیز,None,رقعی,None,None,None,None,[]
8,کتاب سقف تالار آیینه عمارت نارنجستان شیراز,Saghf-e Talar-e Ayieneh,1600,0.0,3.35,کانون پرورش فکری کودکان و نوجوان,هادی سیف,27986,978-9643911140,1390,شومیز,None,رحلی,None,None,None,None,[]
9,کتاب فناوری در سال 2021,"The Year in Tech, 2021",41500,0.0,3.01,راه پرداخت,هاروارد بیزینس ریویو,64447,978-6229736944,116,1399,None,رقعی,None,None,None,None,[]


In [26]:
file_path = "data.csv"

tableOfData.to_csv(file_path, index=False, encoding='utf-8')